关于构建论文的打分系统，以及推荐系统的促钱思考
        a. 拿到文章的摘要，和结论
        b. RAG，检索上下文，基于本文拿到以下内容；
            i. 它核心想要解决什么问题？
            ii. 核心技术贡献？
            iii. 这是不是一个新的思考和方法
            iv. 与当前SOTA对比效果
            v. 本文的作者和团队的技术影响力如何 - 这个需要结合网络搜索
        c. 给大模型推理，给出几个评分指标和打分，推荐理由 ==》打分高于8分的，进入我们的推荐名单
        d. 对抗：
            i. 网络搜索：“它核心想要解决什么问题？”这个问题最近的sota 研究是什么
            ii. 基于以上声明，和网络检索，汇总这些信息，
            iii. 让大模型再来推理一下这个评分是否正确，允许它修正， 并给出修正原因
                1) 大模型可以分为几个专家组？ 
                    a) Ai 算法的专家，
                    b) 计算体协结构的专家，
                    c) 集群和网络互联的专家，
                    d) 芯片设计的专家
                    允许这些专家回答说，不相关，不给出评分
                2) 最终，按照得分最高的专家拿出来用

In [78]:
import os
import sys

# Add the parent directory to Python path
sys.path.append(os.path.dirname(os.getcwd()))

In [11]:
# read paper from postgresql
# -*- coding: utf-8 -*-
from database import SessionLocal

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

In [17]:
from models.tasks import ArxivPaper

db = next(get_db())
paper = db.query(ArxivPaper).offset(0).limit(5).all()
for p in paper:
    print(p.id, p.arxiv_id, p.title)
    print(p.summary[:50])
    print(p.authors)
    print('---')

129 http://arxiv.org/abs/2503.15558v1 Cosmos-Reason1: From Physical Common Sense To Embodied Reasoning
Physical AI systems need to perceive, understand, 
[{'name': ' NVIDIA', 'affiliations': []}, {'name': ' :', 'affiliations': []}, {'name': 'Alisson Azzolini', 'affiliations': []}, {'name': 'Hannah Brandon', 'affiliations': []}, {'name': 'Prithvijit Chattopadhyay', 'affiliations': []}, {'name': 'Huayu Chen', 'affiliations': []}, {'name': 'Jinju Chu', 'affiliations': []}, {'name': 'Yin Cui', 'affiliations': []}, {'name': 'Jenna Diamond', 'affiliations': []}, {'name': 'Yifan Ding', 'affiliations': []}, {'name': 'Francesco Ferroni', 'affiliations': []}, {'name': 'Rama Govindaraju', 'affiliations': []}, {'name': 'Jinwei Gu', 'affiliations': []}, {'name': 'Siddharth Gururani', 'affiliations': []}, {'name': 'Imad El Hanafi', 'affiliations': []}, {'name': 'Zekun Hao', 'affiliations': []}, {'name': 'Jacob Huffman', 'affiliations': []}, {'name': 'Jingyi Jin', 'affiliations': []}, {'name': 'Brend

In [ ]:
# 我们取这篇文章作为实验
# https://arxiv.org/pdf/2401.01040
# TOWARDS COGNITIVE AI SYSTEMS: A SURVEY AND PROSPECTIVE ON NEURO-SYMBOLIC AI
# 先下载论文，手动

In [79]:
# 解析论文，生成RAG
def preprocess_pdf(pdf_path):
    import fitz  # PyMuPDF
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        # print(page.number, page.get_text()[:20])
        text += page.get_text()
    return text

text = preprocess_pdf('../../data/pdf/2401.01040v1.pdf')
lines = [line.strip() for line in text.split("\n") if line.strip()]  # 去除空白行
print(lines[:10])

['TOWARDS COGNITIVE AI SYSTEMS: A SURVEY AND PROSPECTIVE ON', 'NEURO-SYMBOLIC AI', 'Zishen Wan 1 Che-Kai Liu * 1 Hanchen Yang * 1 Chaojian Li * 1 Haoran You * 1 Yonggan Fu 1 Cheng Wan 1', 'Tushar Krishna 1 Yingyan (Celine) Lin 1 Arijit Raychowdhury 1', 'ABSTRACT', 'The remarkable advancements in artificial intelligence (AI), primarily driven by deep neural networks, have', 'significantly impacted various aspects of our lives. However, the current challenges surrounding unsustainable', 'computational trajectories, limited robustness, and a lack of explainability call for the development of next-', 'generation AI systems. Neuro-symbolic AI (NSAI) emerges as a promising paradigm, fusing neural, symbolic,', 'and probabilistic approaches to enhance interpretability, robustness, and trustworthiness while facilitating learning']


In [80]:
# 第二次，记录标题行，目的是后续可以按照标题行来把文本内容分为多个chunks
# 可以继续扩展
SECTION_TITLES = {
    "abstract": {"abstract"},
    "introduction": {"introduction", "background", "preliminaries", "preliminary"},
    "related_work": {"related work", "prior work", "literature review", "related studies"},
    "methodology": {"methodology", "methods", "method", "approach", "proposed method", 
                    "model", "architecture", "framework", "algorithm", "system design"},
    "experiment": {"experiment", "experiments", "experimental setup", "experiment setup",
                   "setup", "implementation details", "evaluation", "evaluation setup"},
    "ablation":{"ablation","ablation study"},
    "results": {"results", "performance", "findings", "observations", "empirical results"},
    "discussion": {"discussion", "analysis", "interpretation"},
    "summary":{"summary"},
    "conclusion": {"conclusion", "final remarks", "closing remarks"},
    "limitations":{"limitations"},
    "acknowledgments": {"acknowledgments", "acknowledgements", "funding", "author contributions"},
    "references": {"references", "bibliography", "cited works"},
    "appendix": {"appendix", "supplementary material", "supplementary", "additional materials"}
}

# 论文拆分参数
MAX_LINE_PER_CHUNK = 25  # 每个 chunk 最大行数
OVERLAP_LINES = 5  # 上下文重叠行数
REFERENCE_BLOCK_SIZE = 8  # 参考文献块大小（5~10行）


In [81]:
import re

def detect_section_titles(lines):
    """检测标题行，返回标题索引"""
    title_indices = {}
    section_num = 0 # 对于 summary、limitation 这类标题，可能在多个位置出现，需要全部保留。加一个编码，保证独立
    for i, line in enumerate(lines):
        clean_line = re.sub(r"[^a-zA-Z\s]", "", line).strip().lower()
        for section, keywords in SECTION_TITLES.items():
            if any(re.match(rf"^\s*(\d+([\.\-）]\d+)*[\.\-）]*\s*)?{re.escape(kw)}\s*$", clean_line, re.IGNORECASE) for kw in keywords):
                if (section in title_indices):
                    title_indices[section+"_"+str(section_num)] = i
                    section_num += 1
                else:
                    title_indices[section] = i
                break
    return title_indices

In [82]:
title_indices = detect_section_titles(lines)  # 2. 识别标题
print(title_indices)

abstract_index = title_indices.get("abstract", None)  # 获取 Abstract 标题的索引
print(abstract_index)

{'abstract': 4, 'introduction': 15, 'conclusion': 544, 'acknowledgments': 552, 'references': 557}
4


In [83]:
from collections import defaultdict

def split_chunks_by_title(lines, title_indices):
    """按照标题行分块"""
    chunks = defaultdict(list)
    sorted_sections = sorted(title_indices.items(), key=lambda x: x[1])  # 按出现顺序排序

    for idx, (section, start_idx) in enumerate(sorted_sections):
        end_idx = sorted_sections[idx + 1][1] if idx + 1 < len(sorted_sections) else len(lines)
        chunks[section] = lines[start_idx:end_idx]
    
    return chunks

In [84]:
section_chunks = split_chunks_by_title(lines, title_indices)  # 3. 按标题拆分
print(section_chunks)

defaultdict(<class 'list'>, {'abstract': ['ABSTRACT', 'The remarkable advancements in artificial intelligence (AI), primarily driven by deep neural networks, have', 'significantly impacted various aspects of our lives. However, the current challenges surrounding unsustainable', 'computational trajectories, limited robustness, and a lack of explainability call for the development of next-', 'generation AI systems. Neuro-symbolic AI (NSAI) emerges as a promising paradigm, fusing neural, symbolic,', 'and probabilistic approaches to enhance interpretability, robustness, and trustworthiness while facilitating learning', 'from much less data. Recent NSAI systems have demonstrated great potential in collaborative human-AI scenarios', 'with reasoning and cognitive capabilities. In this paper, we provide a systematic review of recent progress in', 'NSAI and analyze the performance characteristics and computational operators of NSAI models. Furthermore, we', 'discuss the challenges and potential

In [85]:
# 获取 Abstract 之前的所有内容（作为作者信息）
abstract_index = title_indices.get("abstract", None)  # 获取 Abstract 标题的索引
if abstract_index is not None:
    author_info_lines = lines[:abstract_index]  # 从文档开头到 Abstract 之前的所有行
else:
    author_info_lines = lines[:10]  # 兜底逻辑，取前10行作为作者信息
print(author_info_lines)

['TOWARDS COGNITIVE AI SYSTEMS: A SURVEY AND PROSPECTIVE ON', 'NEURO-SYMBOLIC AI', 'Zishen Wan 1 Che-Kai Liu * 1 Hanchen Yang * 1 Chaojian Li * 1 Haoran You * 1 Yonggan Fu 1 Cheng Wan 1', 'Tushar Krishna 1 Yingyan (Celine) Lin 1 Arijit Raychowdhury 1']


In [86]:
def split_text_with_titles(section_chunks, max_line_per_chunk=25, overlap_lines=5):
    """
    按 Section 拆分文本，并进行 Chunk 切分，考虑标题标记、最大行数和重叠行数。

    :param section_chunks: dict, 以 section title 为 key，存储对应的文本行
    :param max_line_per_chunk: int, 每个 chunk 最大行数
    :param overlap_lines: int, 允许的上下文重叠行数
    :return: List[str]，每个 chunk 作为字符串存入列表
    """
    text_chunks = []
    
    for section, content_lines in section_chunks.items():
        print(f'=debug=processing section: {section}, with line amount: {len(content_lines)}\n')
        if section in {"authors", "acknowledgments","references","appendix"}:  # 跳过作者, 感谢，附录和参考文献， 只提取正文，用于RAG 
            print("=debug=skiped")
            continue  
        
        current_chunk = []
        for i, line in enumerate(content_lines):
            current_chunk.append(line)

            # 当达到 max_line_per_chunk 时，存储当前 chunk，并留出 overlap
            if len(current_chunk) >= max_line_per_chunk or i == len(content_lines) - 1:
                text_chunks.append("\n".join(current_chunk).strip())

                # 重叠部分：保留最后 overlap_lines 行作为下一个 chunk 的起点
                current_chunk = current_chunk[-overlap_lines:] if overlap_lines > 0 else []
    
    return text_chunks

In [87]:
processed_chunks = split_text_with_titles(section_chunks,25,5)
print(len(processed_chunks))
print(processed_chunks[0:2])

=debug=processing section: abstract, with line amount: 11

=debug=processing section: introduction, with line amount: 529

=debug=processing section: conclusion, with line amount: 8

=debug=processing section: acknowledgments, with line amount: 5

=debug=skiped
=debug=processing section: references, with line amount: 155

=debug=skiped
29
['ABSTRACT\nThe remarkable advancements in artificial intelligence (AI), primarily driven by deep neural networks, have\nsignificantly impacted various aspects of our lives. However, the current challenges surrounding unsustainable\ncomputational trajectories, limited robustness, and a lack of explainability call for the development of next-\ngeneration AI systems. Neuro-symbolic AI (NSAI) emerges as a promising paradigm, fusing neural, symbolic,\nand probabilistic approaches to enhance interpretability, robustness, and trustworthiness while facilitating learning\nfrom much less data. Recent NSAI systems have demonstrated great potential in collaborat

In [88]:
import json
from ollama import Client

# 初始化 Ollama 客户端，确保连接到本地 Ollama 服务器
client = Client(host='http://localhost:11434')

# 3. 生成嵌入向量（调用 Ollama）
def get_embedding(text):
    response = client.embeddings(model='nomic-embed-text', prompt=text)
    return response.get('embedding', [])

# 4. 生成 {原文: 向量} 字典
embeddings_list =[]
for chunk in processed_chunks:
    embeddings_list.append([chunk, get_embedding(chunk)])

print(embeddings_list[0][1])
print(len(embeddings_list[0][1]))

[-0.22683027386665344, 2.156106948852539, -2.4321091175079346, -0.5241323113441467, 0.8876791000366211, -0.35857242345809937, 0.8799885511398315, -0.10043108463287354, -1.159690499305725, 1.3548728227615356, 0.4510385990142822, 0.6308646202087402, 1.4287934303283691, 1.0604209899902344, -0.1210523322224617, 0.27044039964675903, -0.630506157875061, 0.7107692360877991, -0.007955603301525116, 0.09365356713533401, -0.5294976830482483, -0.2874680161476135, -0.3762367367744446, -0.31868690252304077, 1.6617238521575928, -0.3259769678115845, -0.09007053822278976, -0.14662739634513855, -0.2622408866882324, 0.12540549039840698, 1.2255455255508423, 0.10081381350755692, 0.287771999835968, -0.7269757390022278, -1.1665589809417725, -0.4588192403316498, 0.014281193725764751, -0.314519464969635, 0.635515570640564, 0.8069763779640198, -0.2747404873371124, 0.6089200973510742, -0.5188892483711243, -0.6254041194915771, 0.08473111689090729, 0.18313245475292206, 0.8017299771308899, -0.8238663077354431, 0.72

In [89]:
print(len(processed_chunks))
print(len(embeddings_list))
print(embeddings_list[0][0])
print(embeddings_list[0][1])

29
29
ABSTRACT
The remarkable advancements in artificial intelligence (AI), primarily driven by deep neural networks, have
significantly impacted various aspects of our lives. However, the current challenges surrounding unsustainable
computational trajectories, limited robustness, and a lack of explainability call for the development of next-
generation AI systems. Neuro-symbolic AI (NSAI) emerges as a promising paradigm, fusing neural, symbolic,
and probabilistic approaches to enhance interpretability, robustness, and trustworthiness while facilitating learning
from much less data. Recent NSAI systems have demonstrated great potential in collaborative human-AI scenarios
with reasoning and cognitive capabilities. In this paper, we provide a systematic review of recent progress in
NSAI and analyze the performance characteristics and computational operators of NSAI models. Furthermore, we
discuss the challenges and potential future directions of NSAI from both system and architectural pe

In [90]:
from tqdm import tqdm
import time

data = []
arxiv_id = "2401.01040"
retries = 3
for i, line in enumerate(tqdm(processed_chunks, desc="Creating embeddings")):
    for attempt in range(retries):
        try:
            data.append({"arxiv_id": arxiv_id, "id": i, "vector": embeddings_list[i][1], "text": embeddings_list[i][0]})
            break
        except Exception as e:
            print(f"Error on line '{line}': {e}. Retrying {attempt + 1}/{retries}...")
            time.sleep(3*(attempt+1))  # 重试前等待 2 秒
            if attempt == retries - 1:
                print(f"Failed to process line '{line}' after {retries} attempts.")
                # 记录错误信息（可选）
                # data.append({"id": i, "vector": None, "text": line, "error": str(e)})

Creating embeddings: 100%|██████████| 29/29 [00:00<00:00, 149428.52it/s]


In [91]:
print(len(data))
print(data[0])

29
{'arxiv_id': '2401.01040', 'id': 0, 'vector': [-0.22683027386665344, 2.156106948852539, -2.4321091175079346, -0.5241323113441467, 0.8876791000366211, -0.35857242345809937, 0.8799885511398315, -0.10043108463287354, -1.159690499305725, 1.3548728227615356, 0.4510385990142822, 0.6308646202087402, 1.4287934303283691, 1.0604209899902344, -0.1210523322224617, 0.27044039964675903, -0.630506157875061, 0.7107692360877991, -0.007955603301525116, 0.09365356713533401, -0.5294976830482483, -0.2874680161476135, -0.3762367367744446, -0.31868690252304077, 1.6617238521575928, -0.3259769678115845, -0.09007053822278976, -0.14662739634513855, -0.2622408866882324, 0.12540549039840698, 1.2255455255508423, 0.10081381350755692, 0.287771999835968, -0.7269757390022278, -1.1665589809417725, -0.4588192403316498, 0.014281193725764751, -0.314519464969635, 0.635515570640564, 0.8069763779640198, -0.2747404873371124, 0.6089200973510742, -0.5188892483711243, -0.6254041194915771, 0.08473111689090729, 0.183132454752922

In [92]:
# Define the collection schema
from pymilvus import Collection, CollectionSchema, DataType, FieldSchema, MilvusClient

milvus_client = MilvusClient("milvus_demo.db")

In [93]:
dim = len(embeddings_list[0][1])  # Get the dimension of your embeddings
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True),
    FieldSchema(name="arxiv_id", dtype=DataType.VARCHAR, max_length=32),
    FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=65535),
    FieldSchema(name="vector", dtype=DataType.FLOAT_VECTOR, dim=dim)
]
schema = CollectionSchema(fields=fields, description="paper_embeddings")

# Create collection
collection_name = "paper_embeddings"

if milvus_client.has_collection(collection_name=collection_name):
    milvus_client.drop_collection(collection_name=collection_name)
milvus_client.create_collection(
    collection_name=collection_name,
    schema=schema)

# Create an IVF_FLAT index for the embedding field
field_name = "vector"

# Define index parameters
index_params = [{
    "field_name": field_name,
    "index_type": "IVF_FLAT",
    "metric_type": "IP",
    "params": {"nlist": 128}  # nlist: number of clusters (adjust as needed)
}]

# Create the index
milvus_client.create_index(
    collection_name=collection_name,
    index_params=index_params
)


In [94]:
milvus_client.insert(collection_name=collection_name, data=data)

{'insert_count': 29, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28], 'cost': 0}

In [95]:
# search
my_question = "what is the abstract?"
emb_question = get_embedding(my_question)
print(emb_question)

[0.5097389221191406, 0.7643529176712036, -2.717076301574707, -2.796359062194824, 0.48820894956588745, 0.17529773712158203, -0.0026921313256025314, -0.18672415614128113, -0.5681865215301514, -0.019207902252674103, 0.1707848310470581, 0.14132288098335266, 2.8740363121032715, 1.5528528690338135, 1.0321707725524902, -1.5478198528289795, -0.7940031886100769, -1.550112247467041, 0.8407857418060303, -0.0866343230009079, -0.527118444442749, -0.08706706762313843, -0.018099743872880936, -0.7683690190315247, 2.0643463134765625, 0.1448265016078949, 1.4359667301177979, -0.677114725112915, -1.0343513488769531, 0.6570568084716797, 0.7260560393333435, 0.46517467498779297, 0.5738341808319092, -0.9589502215385437, -0.484345018863678, 0.10073967278003693, -0.009476006031036377, 2.3630242347717285, 0.1101347953081131, -0.594012975692749, -0.273466020822525, 1.2196016311645508, -0.3849544823169708, -0.744173526763916, 0.00511213019490242, 1.1873400211334229, 0.3840443193912506, -0.2720259428024292, 0.04911

In [96]:
my_question = "what is the abstract?"
emb_question = get_embedding(my_question)

search_res = milvus_client.search(
    collection_name=collection_name,
    data=[
        emb_question
    ],  # Use the `emb_text` function to convert the question to an embedding vector
    limit=10,  # Return top 3 results
    search_params={"metric_type": "IP", "params": {"nprobe": 10}},  # Inner product distance
    output_fields=["text"],  # Return the text field
)

retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4))

[
    [
        "in Tab. 1. We elaborate each of these paradigms below. Ad-\nditionally, Tab. 2 provides examples of several underlying\noperations based on the categorization in Tab. 1.\nSymbolic[Neuro] refers to an intelligent system that em-\npowers symbolic reasoning with the statistical learning ca-\npabilities of NNs. These systems typically consist of a com-\nprehensive symbolic problem solver that includes loosely-\ncoupled neural subroutines for statistical learning. Exam-\nples include DeepMind\u2019s AlphaGo (Silver et al., 2017) and\nAlphaZero (Zhang & Yu, 2020), which use Monte-Carlo\nTree Search (MCTS) as the symbolic solver and NN state\nestimators for learning statistical patterns.\nTable 1. Survey of recent NSAI algorithms into five groups, with a\nsummary of their underlying operations and vector formats.\nCategory\nNSAI Algorithm\nUnderlying Operation\nIf Vector\nSymbolic[Neuro]\nAlphaGo (Silver et al., 2017)\nNN, MCTS\nVector\nNeuro|Symbolic\nNVSA (Hersche et al., 2

In [ ]:
# 通过上面的实验，我发现RAG的召回效果不好。准确率有点差。
# 重新测试，直接将文章的核心信息，给大模型来分析
# 如果将全文丢给大模型，这样的话token 消耗又太大了--> 
# TODO 未来再考虑如何结合token

In [135]:
# 接下来，我们直接全文让ai 来总结关键问题
# 构建一个全文文本：
# print('\n'.join(section_chunks.get('abstract')))
full_text_lines=[]
for key in title_indices:
    print(key)
    full_text_lines.append('\n'.join(section_chunks.get(key)))
    if key.strip().lower() == 'conclusion' or key.strip().lower() == 'references':
        break
# print(full_text[:500])
print(len(full_text_lines))
full_text = '\n'.join(full_text_lines)
print(f'totoal words: {len(full_text)}')
print(full_text[:1000])

abstract
introduction
conclusion
3
totoal words: 22585
ABSTRACT
The remarkable advancements in artificial intelligence (AI), primarily driven by deep neural networks, have
significantly impacted various aspects of our lives. However, the current challenges surrounding unsustainable
computational trajectories, limited robustness, and a lack of explainability call for the development of next-
generation AI systems. Neuro-symbolic AI (NSAI) emerges as a promising paradigm, fusing neural, symbolic,
and probabilistic approaches to enhance interpretability, robustness, and trustworthiness while facilitating learning
from much less data. Recent NSAI systems have demonstrated great potential in collaborative human-AI scenarios
with reasoning and cognitive capabilities. In this paper, we provide a systematic review of recent progress in
NSAI and analyze the performance characteristics and computational operators of NSAI models. Furthermore, we
discuss the challenges and potential future directi

In [ ]:
from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="gpt-4o-mini",
    input="Write a one-sentence bedtime story about a unicorn."
)

print(response)

Response(id='resp_67ed9d3558ac8191b78381548f1bb9700cecd877f7cfa057', created_at=1743625525.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-mini-2024-07-18', object='response', output=[ResponseOutputMessage(id='msg_67ed9d35ecf88191bc71369a8cc515230cecd877f7cfa057', content=[ResponseOutputText(annotations=[], text='As the stars twinkled like diamonds, Luna the unicorn danced through the enchanted forest, sprinkling stardust over sleepy flowers and whispering sweet dreams into the night.', type='output_text')], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, max_output_tokens=None, previous_response_id=None, reasoning=Reasoning(effort=None, generate_summary=None), status='completed', text=ResponseTextConfig(format=ResponseFormatText(type='text')), truncation='disabled', usage=ResponseUsage(input_tokens=18, input_tokens_details=InputTokensDetails(cached_tokens=0

In [151]:
reviewer_instruction='''你现在一位<AI 算法和理论创新>领域的专家，熟悉学术界和工业界的最新研究成果。我现在给你一篇文章，包括文章的核心信息。
现在，要求你从以下几个维度来打分，分数从1到10。10分代表最好：
（1） 技术创新性 (Technical Innovation)： 考察其方法相对于已有方案的新颖程度。例如，判断论文是否引入了从未有文献提出过的新机制，或是将以往被视为必要的组件替换为全新的架构。如果一项工作开辟了新方向、引发社区将注意力转向新的研究范式，那么它在技术创新性维度应获得高分，可以在8分以上。
（2）性能提升幅度 (Performance Improvement)：查看其在领域公认基准数据集上的实验结果表格，将其与之前发表的最佳结果进行比较。如果论文报告的主要指标有大幅领先，则可判断为高分。如果提升有限甚至没有超越已有结果，则在此维度评分不高。此外，可关注提升是否在多个任务/数据集上复现，还是仅局限于单一场景。真正颠覆性的成果往往在不同测试上都表现优异，体现出方法的普适性能和强大性能增益。
（3）理论或工程简洁性 (Theoretical/Engineering Simplicity & Efficiency)：考察其方法是否减少了计算复杂度或工程难度。例如，关注算法的时间复杂度是否降低，模型参数/内存占用是否减少，训练所需算力或时间是否更少。可量化指标包括：与前代方法相比，加速比率、参数减少百分比、模型大小与性能比值等。如果论文提供了与基准方法在计算耗时或资源使用上的对比，那是衡量简洁高效的重要依据。高分论文常能在保持或提高性能的同时，简化模型结构或提高训练推理效率，这样的工作在应用中更具颠覆性价值。
（4）可复用性与影响力 (Reusability & Impact)：考察该工作的通用模块或框架性质。例如，论文是否提供了一个易于迁移到其他任务的模型结构或算法。如果一项方法很快融入主流工具链并成为众多研究的基础，则它的可复用性与影响力应打高分。相反，如果方法过于定制或复杂，鲜有他人采用，则此维度得分会较低。
（5）作者与机构权威度 (Author & Institutional Authority)：查阅作者的背景（例如Google Scholar页面、实验室网站）以了解他们以往的研究成绩。如果作者来自公认的顶级研究单位或曾发表过有影响力的相关工作，则权威度评分可相应提高。还可考虑论文是否有大咖参与（如知名教授、业界著名研究员），以及机构是否拥有强大的研究传统（比如是否曾培养出许多突破性成果）。

最后，请你给出一个建议，是否推荐阅读这篇文章，yes or no, 以及谈一谈你的原因。 如果是一个信息汇总，且没有带来很多额外价值信息, 你可以大胆地打低分，并且表达不推荐。

请你将最总结果按照以下json 格式输出，严格按照json格式，请不要加额外的信息。
Example of your outputs, the score is just a example, not a reference.
{
  "score": [
    {
      "innovation": 7,
      "reason": "The paper systematically reviews and categorizes ….."
    },
    {
      "performance": 5,
      "reason": "As a survey paper, it does not present new experimental results ….."
    },
    {
      "simplicity": 6,
      "reason": "The paper discusses the computational characteristics and potential ….."
    },
    {
      "reusability": 8,
      "reason": "By categorizing ….."
    },
    {
      "authority": 7,
      "reason": " a reputable institution known for its contributions to …."
    }
  ],
  "recommend": "yes",
  "reason": "The paper offers a comprehensive overview of ….."
  "who_should_read": "beginner… "
}
'''

In [152]:
paper_context = '''
论文名称：“TOWARDS COGNITIVE AI SYSTEMS: A SURVEY AND PROSPECTIVE ON NEURO-SYMBOLIC AI”
摘要：“The remarkable advancements in artificial intelligence (AI), primarily driven by deep neural networks, have significantly impacted various aspects of our lives. However, the current challenges surrounding unsustainable computational trajectories, limited robustness, and a lack of explainability call for the development of nextgeneration AI systems. Neuro-symbolic AI (NSAI) emerges as a promising paradigm, fusing neural, symbolic, and probabilistic approaches to enhance interpretability, robustness, and trustworthiness while facilitating learning from much less data. Recent NSAI systems have demonstrated great potential in collaborative human-AI scenarios with reasoning and cognitive capabilities. In this paper, we provide a systematic review of recent progress in NSAI and analyze the performance characteristics and computational operators of NSAI models. Furthermore, we discuss the challenges and potential future directions of NSAI from both system and architectural perspectives.”
结论：“NSAI is an emerging paradigm for next-generation efficient, robust, explainable, and cognitive AI systems. This paper systematically reviews recent NSAI algorithms, characterizes their system performance, analyzes their workload operators, and identifies the challenges and opportunities towards fullfiling next-generation NSAI systems.”
'''

In [ ]:
response = client.responses.create(
    model="gpt-4o-mini",
    instructions=reviewer_instruction,
    input=paper_context
)

print(response)

Response(id='resp_67ed9e986fa081919c5123533ff05d500a172234a88c2e2d', created_at=1743625880.0, error=None, incomplete_details=None, instructions='你现在一位<AI 算法和理论创新>领域的专家，熟悉学术界和工业界的最新研究成果。我现在给你一篇文章，包括文章的核心信息。\n现在，要求你从以下几个维度来打分，分数从1到10。10分代表最好：\n（1） 技术创新性 (Technical Innovation)： 考察其方法相对于已有方案的新颖程度。例如，判断论文是否引入了从未有文献提出过的新机制，或是将以往被视为必要的组件替换为全新的架构。如果一项工作开辟了新方向、引发社区将注意力转向新的研究范式，那么它在技术创新性维度应获得高分，可以在8分以上。\n（2）性能提升幅度 (Performance Improvement)：查看其在领域公认基准数据集上的实验结果表格，将其与之前发表的最佳结果进行比较。如果论文报告的主要指标有大幅领先，则可判断为高分。如果提升有限甚至没有超越已有结果，则在此维度评分不高。此外，可关注提升是否在多个任务/数据集上复现，还是仅局限于单一场景。真正颠覆性的成果往往在不同测试上都表现优异，体现出方法的普适性能和强大性能增益。\n（3）理论或工程简洁性 (Theoretical/Engineering Simplicity & Efficiency)：考察其方法是否减少了计算复杂度或工程难度。例如，关注算法的时间复杂度是否降低，模型参数/内存占用是否减少，训练所需算力或时间是否更少。可量化指标包括：与前代方法相比，加速比率、参数减少百分比、模型大小与性能比值等。如果论文提供了与基准方法在计算耗时或资源使用上的对比，那是衡量简洁高效的重要依据。高分论文常能在保持或提高性能的同时，简化模型结构或提高训练推理效率，这样的工作在应用中更具颠覆性价值。\n（4）可复用性与影响力 (Reusability & Impact)：考察该工作的通用模块或框架性质。例如，论文是否提供了一个易于迁移到其他任务的模型结构或算法。如果一项方法很快融入主流工具链并成为众多研究的基础，则它的可复用性与影响力应打高分。相反，如果方法过于定制或复

In [148]:
response.output_text

'{\n  "score": [\n    {\n      "innovation": 6,\n      "reason": "The paper surveys the current advancements in neuro-symbolic AI, but does not propose a novel mechanism or framework."\n    },\n    {\n      "performance": 5,\n      "reason": "While the paper reviews existing NSAI systems, it lacks new experimental results to support claims of performance improvements."\n    },\n    {\n      "simplicity": 7,\n      "reason": "The paper discusses the computational characteristics, which may suggest simpler approaches but does not provide quantifiable measures of complexity reduction."\n    },\n    {\n      "reusability": 6,\n      "reason": "The review lays groundwork for future work in NSAI, but the immediate applicability of the discussed methods may be limited."\n    },\n    {\n      "authority": 8,\n      "reason": "The authors are associated with reputable institutions known for contributions to AI research, adding credibility to the findings."\n    }\n  ],\n  "recommend": "no",\n  

In [149]:
# 让我们再问关键的几个问题
info_summery_assistant = '''请根据以下问题，提供详细的回答，并确保答案以JSON格式返回。每个问题后紧跟其答案：
{
"core_problem": "1. 这篇文章试图解决的核心问题是什么？",
"technical_contributions": "2. 该论文的主要技术贡献是什么？",
"innovations_and_proposals": [
"3. 该论文提出了哪些创新点或建议？"
],
"sota_comparison": "4. 该论文是否与当前最先进的研究成果进行了比较？如果有，比较的结论是什么？",
"authors_and_affiliations": {
"authors": [
"5. 论文的作者是谁？"
],
"institution": "6. 他们属于哪个机构？"
},
"influence_assessment": "7. 请评估作者和其所属机构在学术界和工业界的影响力。"
}

请用具体的信息替换JSON中的每个问题，并确保生成的JSON格式正确。'''

In [153]:
response = client.responses.create(
    model="gpt-4o-mini",
    instructions=info_summery_assistant,
    input=paper_context
)

print(response)

Response(id='resp_67eda38bbfc48191bb56d98c10ea43c80474a926a37efb41', created_at=1743627147.0, error=None, incomplete_details=None, instructions='请根据以下问题，提供详细的回答，并确保答案以JSON格式返回。每个问题后紧跟其答案：\n{\n"core_problem": "1. 这篇文章试图解决的核心问题是什么？",\n"technical_contributions": "2. 该论文的主要技术贡献是什么？",\n"innovations_and_proposals": [\n"3. 该论文提出了哪些创新点或建议？"\n],\n"sota_comparison": "4. 该论文是否与当前最先进的研究成果进行了比较？如果有，比较的结论是什么？",\n"authors_and_affiliations": {\n"authors": [\n"5. 论文的作者是谁？"\n],\n"institution": "6. 他们属于哪个机构？"\n},\n"influence_assessment": "7. 请评估作者和其所属机构在学术界和工业界的影响力。"\n}\n\n请用具体的信息替换JSON中的每个问题，并确保生成的JSON格式正确。', metadata={}, model='gpt-4o-mini-2024-07-18', object='response', output=[ResponseOutputMessage(id='msg_67eda38c90c48191b52181b3c2c57e8f0474a926a37efb41', content=[ResponseOutputText(annotations=[], text='```json\n{\n  "core_problem": "1. 这篇文章试图解决的核心问题是什么？: 文章关注人工智能领域中当前AI系统面临的挑战，如计算成本无法持续、稳定性不足和缺乏可解释性，并提出了开发下一代AI系统的必要性。",\n  "technical_contributions": "2. 该论文的主要技术贡献是什么？: 论文系统性地回顾了神经符号AI（NSAI）的进展，分析了

In [154]:
response.output_text

'```json\n{\n  "core_problem": "1. 这篇文章试图解决的核心问题是什么？: 文章关注人工智能领域中当前AI系统面临的挑战，如计算成本无法持续、稳定性不足和缺乏可解释性，并提出了开发下一代AI系统的必要性。",\n  "technical_contributions": "2. 该论文的主要技术贡献是什么？: 论文系统性地回顾了神经符号AI（NSAI）的进展，分析了NSAI模型的性能特征和计算操作，为改进AI的可解释性、稳健性和信任度提供了理论基础和实际实施建议。",\n  "innovations_and_proposals": [\n    "3. 该论文提出了哪些创新点或建议？: 论文中提出通过融合神经网络、符号处理和概率模型来增强AI系统的表现，探讨了NSAI在数据稀缺情况下的学习能力，并识别出未来研究的潜在方向。"\n  ],\n  "sota_comparison": "4. 该论文是否与当前最先进的研究成果进行了比较？如果有，比较的结论是什么？: 是的，论文对现有的最先进技术进行了比较，指出了NSAI在协作人机场景中展现出更好的推理和认知能力，从而在当前AI技术中具有显著优势。",\n  "authors_and_affiliations": {\n    "authors": [\n      "5. 论文的作者是谁？: 具体作者信息未提供，通常在论文中列出。"\n    ],\n    "institution": "6. 他们属于哪个机构？: 具体机构信息未提供，通常在论文中列出。"\n  },\n  "influence_assessment": "7. 请评估作者和其所属机构在学术界和工业界的影响力。: 由于缺少具体的作者和机构信息，无法进行权威评估，但一般来说，做出NSAI贡献的学者和机构在AI研究领域普遍具有较高的学术声誉和应用影响力。"\n}\n```'

In [155]:
# 获取到某1个或者多个话题的最新学术进展，这个定时更新的事实标准，实际上是可以缓存，从我们自己的系统里面拿，不用每次请求api. 数据库内容定时刷新
sota_info ='''为了更好地帮助你判断文章的价值，我检索了当前的state of the art 信息供你参考：Neuro-Symbolic AI integrates the learning capabilities of neural networks with the logical reasoning strengths of symbolic AI, aiming to develop systems with enhanced cognitive abilities. The latest research directions and advancements in this field include: 
1. Knowledge Representation and Integration: Developing methods to combine symbolic knowledge with neural networks to enhance AI systems’ understanding and reasoning capabilities. For example, integrating knowledge graphs with deep learning models to perform complex reasoning tasks.
 2. Combining Learning and Reasoning: Exploring new neural network architectures and training algorithms that incorporate symbolic reasoning, thereby improving model interpretability and generalization.  
3. Meta-Cognition: Investigating ways to equip AI systems with self-monitoring and self-adjusting abilities, enhancing their autonomy and adaptability.
 4. Verification and Validation (V&V) Methods: Developing new techniques to ensure the reliability and safety of neuro-symbolic AI systems.
 5. Hardware Architecture Optimization: Analyzing the workload characteristics of neuro-symbolic AI to design efficient hardware architectures that support complex neuro-symbolic computations.  
6. Application of Foundation Models: Utilizing large pre-trained models (e.g., GPT) to enhance neuro-symbolic learning and reasoning tasks, reducing the dependence on extensive labeled data.

These research directions aim to overcome current AI system challenges in interpretability, robustness, and data efficiency, propelling the development of next-generation cognitive AI systems.
'''
final_query ='\n'.join([paper_context,sota_info])
print(final_query)


论文名称：“TOWARDS COGNITIVE AI SYSTEMS: A SURVEY AND PROSPECTIVE ON NEURO-SYMBOLIC AI”
摘要：“The remarkable advancements in artificial intelligence (AI), primarily driven by deep neural networks, have significantly impacted various aspects of our lives. However, the current challenges surrounding unsustainable computational trajectories, limited robustness, and a lack of explainability call for the development of nextgeneration AI systems. Neuro-symbolic AI (NSAI) emerges as a promising paradigm, fusing neural, symbolic, and probabilistic approaches to enhance interpretability, robustness, and trustworthiness while facilitating learning from much less data. Recent NSAI systems have demonstrated great potential in collaborative human-AI scenarios with reasoning and cognitive capabilities. In this paper, we provide a systematic review of recent progress in NSAI and analyze the performance characteristics and computational operators of NSAI models. Furthermore, we discuss the challenges and p

In [156]:
response = client.responses.create(
    model="gpt-4o-mini",
    instructions=reviewer_instruction,
    input=final_query
)

print(response)

Response(id='resp_67eda5167ae88191abcb9a9d36522f1c03b56f0d1affccc7', created_at=1743627542.0, error=None, incomplete_details=None, instructions='你现在一位<AI 算法和理论创新>领域的专家，熟悉学术界和工业界的最新研究成果。我现在给你一篇文章，包括文章的核心信息。\n现在，要求你从以下几个维度来打分，分数从1到10。10分代表最好：\n（1） 技术创新性 (Technical Innovation)： 考察其方法相对于已有方案的新颖程度。例如，判断论文是否引入了从未有文献提出过的新机制，或是将以往被视为必要的组件替换为全新的架构。如果一项工作开辟了新方向、引发社区将注意力转向新的研究范式，那么它在技术创新性维度应获得高分，可以在8分以上。\n（2）性能提升幅度 (Performance Improvement)：查看其在领域公认基准数据集上的实验结果表格，将其与之前发表的最佳结果进行比较。如果论文报告的主要指标有大幅领先，则可判断为高分。如果提升有限甚至没有超越已有结果，则在此维度评分不高。此外，可关注提升是否在多个任务/数据集上复现，还是仅局限于单一场景。真正颠覆性的成果往往在不同测试上都表现优异，体现出方法的普适性能和强大性能增益。\n（3）理论或工程简洁性 (Theoretical/Engineering Simplicity & Efficiency)：考察其方法是否减少了计算复杂度或工程难度。例如，关注算法的时间复杂度是否降低，模型参数/内存占用是否减少，训练所需算力或时间是否更少。可量化指标包括：与前代方法相比，加速比率、参数减少百分比、模型大小与性能比值等。如果论文提供了与基准方法在计算耗时或资源使用上的对比，那是衡量简洁高效的重要依据。高分论文常能在保持或提高性能的同时，简化模型结构或提高训练推理效率，这样的工作在应用中更具颠覆性价值。\n（4）可复用性与影响力 (Reusability & Impact)：考察该工作的通用模块或框架性质。例如，论文是否提供了一个易于迁移到其他任务的模型结构或算法。如果一项方法很快融入主流工具链并成为众多研究的基础，则它的可复用性与影响力应打高分。相反，如果方法过于定制或复

In [157]:
response.output_text

'{\n  "score": [\n    {\n      "innovation": 6,\n      "reason": "The paper presents a systematic review of NSAI, though it does not introduce new mechanisms but discusses existing advancements."\n    },\n    {\n      "performance": 5,\n      "reason": "The focus is on a review rather than new experimental results, limiting performance comparison with state-of-the-art systems."\n    },\n    {\n      "simplicity": 7,\n      "reason": "The paper effectively outlines the performance characteristics and operators of NSAI but does not provide detailed efficiency metrics."\n    },\n    {\n      "reusability": 8,\n      "reason": "It addresses various frameworks and challenges in NSAI, making it relevant for future research and application."\n    },\n    {\n      "authority": 7,\n      "reason": "The authors appear to be from a reputable institution, adding credibility to the review of NSAI advancements."\n    }\n  ],\n  "recommend": "yes",\n  "reason": "The paper offers a comprehensive overv